In [1]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges


In [2]:
# load the Cora dataset
dataset = 'Cora'
path = osp.join('.', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [3]:
# use train_test_split_edges to create neg and positive edges
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)  # can input test and val ratios 
print(data)

/Users/sadiaanjum/miniconda3/envs/py_env/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[2708, 1433], val_pos_edge_index=[2, 263], test_pos_edge_index=[2, 527], train_pos_edge_index=[2, 8976], train_neg_adj_mask=[2708, 2708], val_neg_edge_index=[2, 263], test_neg_edge_index=[2, 527])


In [4]:
data.test_neg_edge_index

tensor([[1025,  920, 2053,  ...,  938, 2105, 1632],
        [2305, 1650, 2104,  ..., 1753, 2258, 1641]])

In [5]:
data.x.shape, data.train_pos_edge_index.shape

(torch.Size([2708, 1433]), torch.Size([2, 8976]))

In [10]:
#initializing the value of z to see what it looks like

z = 0
edge_idx_out = 0
train_logits_out = 0

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 128)
        self.conv2 = GCNConv(128, 64)

    def encode(self):
        x = self.conv1(data.x, data.train_pos_edge_index) # convolution 1
        x = x.relu()
        return self.conv2(x, data.train_pos_edge_index) # convolution 2

    def decode(self, z, pos_edge_index, neg_edge_index): # only pos and neg edges
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) # concatenate pos and neg edges
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product 
        return logits

    def decode_all(self, z): 
        prob_adj = z @ z.t() # get adj NxN
        return (prob_adj > 0).nonzero(as_tuple=False).t() # get predicted edge_list 

In [5]:
#experiment

neg_edge_index = negative_sampling(edge_index=data.train_pos_edge_index, #positive edges
    num_nodes=data.num_nodes, # number of nodes
    num_neg_samples=data.train_pos_edge_index.size(1))

edge_index = torch.cat([data.train_pos_edge_index, neg_edge_index], dim=-1)


print(" Train Positive Edge index: ", data.train_pos_edge_index)
print("negative sampled train edge index : ", neg_edge_index)
print(edge_index)

 Train Positive Edge index:  tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
negative sampled train edge index :  tensor([[ 230, 1915, 1004,  ...,  518,  699, 1119],
        [1031, 2182, 2104,  ..., 1661,  934, 1593]])
tensor([[   0,    0,    0,  ...,  518,  699, 1119],
        [ 633, 1862, 2582,  ..., 1661,  934, 1593]])


In [6]:
# model and optrimizer initialize

model = Net()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [7]:

def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train():
    model.train()

    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index, #positive edges
        num_nodes=data.num_nodes, # number of nodes
        num_neg_samples=data.train_pos_edge_index.size(1)) # number of neg_sample equal to number of pos_edges

    optimizer.zero_grad()
    
    z = model.encode() #encode
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index) # decode
    
    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels) # DIFFERENCE between logits and links 
    loss.backward()
    optimizer.step()

    return loss, z 


@torch.no_grad()
def test():
    model.eval()
    perfs = []
    for prefix in ["val", "test"]:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']

        z = model.encode() # encode train
        link_logits = model.decode(z, pos_edge_index, neg_edge_index) # decode test or val
        link_probs = link_logits.sigmoid() # apply sigmoid
        
        link_labels = get_link_labels(pos_edge_index, neg_edge_index) # get link
        
        perfs.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score
    return perfs


In [8]:
best_val_perf = test_perf = 0
for epoch in range(1, 101):
    train_loss, z  = train()
    val_perf, tmp_test_perf = test()
    if val_perf > best_val_perf:
        best_val_perf = val_perf
        test_perf = tmp_test_perf
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    if epoch % 10 == 0:
        print(log.format(epoch, train_loss, best_val_perf, test_perf))

Epoch: 010, Loss: 0.6849, Val: 0.7233, Test: 0.7365
Epoch: 020, Loss: 0.6366, Val: 0.7233, Test: 0.7365
Epoch: 030, Loss: 0.5606, Val: 0.7480, Test: 0.7764
Epoch: 040, Loss: 0.5111, Val: 0.8285, Test: 0.8460
Epoch: 050, Loss: 0.4771, Val: 0.8587, Test: 0.8692
Epoch: 060, Loss: 0.4688, Val: 0.8590, Test: 0.8886
Epoch: 070, Loss: 0.4678, Val: 0.8623, Test: 0.8870
Epoch: 080, Loss: 0.4593, Val: 0.8666, Test: 0.8932
Epoch: 090, Loss: 0.4510, Val: 0.8753, Test: 0.8996
Epoch: 100, Loss: 0.4447, Val: 0.8825, Test: 0.9051


In [9]:
print(z)

tensor([[ 0.0842,  0.1161, -0.3919,  ..., -0.1930,  0.2329, -0.3036],
        [ 0.2228,  0.0164, -0.0316,  ...,  0.1760, -0.1775, -0.0110],
        [ 0.2553,  0.1192, -0.1332,  ...,  0.1048, -0.1043, -0.0789],
        ...,
        [-0.0891, -0.0584, -0.1147,  ...,  0.0319,  0.1375, -0.1929],
        [ 0.2289,  0.1896, -0.2989,  ..., -0.0774,  0.0509, -0.2078],
        [ 0.1716,  0.2630, -0.3139,  ..., -0.1206,  0.1075, -0.2221]],
       grad_fn=<AddBackward0>)


In [11]:
z[edge_index[0]]

tensor([[ 0.0842,  0.1161, -0.3919,  ..., -0.1930,  0.2329, -0.3036],
        [ 0.0842,  0.1161, -0.3919,  ..., -0.1930,  0.2329, -0.3036],
        [ 0.0842,  0.1161, -0.3919,  ..., -0.1930,  0.2329, -0.3036],
        ...,
        [-0.1536,  0.0089,  0.1703,  ...,  0.0804, -0.0374,  0.1541],
        [-0.1116,  0.2604, -0.1583,  ..., -0.2681,  0.0696, -0.0921],
        [ 0.4288,  0.3826,  0.2403,  ...,  0.0485, -0.2326, -0.2171]],
       grad_fn=<IndexBackward0>)

In [17]:
edge_index[0], edge_index[1]

(tensor([   0,    0,    0,  ...,  518,  699, 1119]),
 tensor([ 633, 1862, 2582,  ..., 1661,  934, 1593]))

In [18]:
z[0], z[633]

(tensor([ 0.0842,  0.1161, -0.3919, -0.3125,  0.0420,  0.2137, -0.1114, -0.0886,
         -0.3080, -0.1099, -0.1253,  0.1736, -0.1266, -0.1786, -0.0658, -0.0413,
         -0.1444,  0.2327,  0.1802,  0.1453,  0.0230, -0.0880,  0.1947, -0.2468,
         -0.0575, -0.0790, -0.0356,  0.1408, -0.0804,  0.3461, -0.0643, -0.0354,
          0.2093, -0.0143,  0.0636, -0.1094, -0.1221, -0.1453, -0.3665, -0.1973,
         -0.1546, -0.2013, -0.1228, -0.2422, -0.0269,  0.2767, -0.3096,  0.4554,
          0.1123,  0.0790,  0.1547, -0.0062,  0.1694,  0.0316,  0.2772, -0.2790,
          0.3547,  0.3170, -0.1324,  0.1133, -0.0635, -0.1930,  0.2329, -0.3036],
        grad_fn=<SelectBackward0>),
 tensor([ 0.0918,  0.0551, -0.2775, -0.2002,  0.0357,  0.1627, -0.0915, -0.0977,
         -0.2301, -0.0721, -0.0968,  0.1218, -0.0708, -0.1569, -0.0285, -0.0191,
         -0.1273,  0.1191,  0.1455,  0.1034,  0.0004, -0.0507,  0.1636, -0.1862,
         -0.0099, -0.0762, -0.0213,  0.0807, -0.0289,  0.2439, -0.0301, 

In [19]:
z[0].shape, z[633].shape

(torch.Size([64]), torch.Size([64]))

In [24]:
z[edge_index[0]], z[edge_index[0]].shape

(tensor([[ 0.0842,  0.1161, -0.3919,  ..., -0.1930,  0.2329, -0.3036],
         [ 0.0842,  0.1161, -0.3919,  ..., -0.1930,  0.2329, -0.3036],
         [ 0.0842,  0.1161, -0.3919,  ..., -0.1930,  0.2329, -0.3036],
         ...,
         [-0.1536,  0.0089,  0.1703,  ...,  0.0804, -0.0374,  0.1541],
         [-0.1116,  0.2604, -0.1583,  ..., -0.2681,  0.0696, -0.0921],
         [ 0.4288,  0.3826,  0.2403,  ...,  0.0485, -0.2326, -0.2171]],
        grad_fn=<IndexBackward0>),
 torch.Size([17952, 64]))

In [22]:
(z[edge_index[0]] * z[edge_index[1]]).shape

torch.Size([17952, 64])

In [25]:
(z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1).shape

torch.Size([17952])

In [26]:
(z[edge_index[0]] * z[edge_index[1]])

tensor([[ 0.0077,  0.0064,  0.1087,  ...,  0.0225,  0.0391,  0.0724],
        [ 0.0092,  0.0240,  0.1551,  ...,  0.0315,  0.0506,  0.0941],
        [ 0.0069,  0.0076,  0.1301,  ...,  0.0276,  0.0462,  0.0758],
        ...,
        [ 0.0422,  0.0005,  0.0248,  ..., -0.0062,  0.0009,  0.0257],
        [-0.0278, -0.0192, -0.0028,  ..., -0.0394, -0.0156, -0.0060],
        [ 0.1128,  0.0070, -0.0023,  ...,  0.0135, -0.0626,  0.1179]],
       grad_fn=<MulBackward0>)

In [15]:
logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)
print(logits)
logits.shape

tensor([ 1.7371,  2.4267,  1.9721,  ...,  1.1782, -0.7469, -0.4176],
       grad_fn=<SumBackward1>)


torch.Size([17952])

In [10]:
z.shape

torch.Size([2708, 64])

In [18]:
z1 = model.encode()

In [20]:
final_edge_index = model.decode_all(z1)

In [21]:
final_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    2,    4,  ..., 2705, 2706, 2707]])

In [23]:
final_edge_index = model.decode_all(z)
final_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    2,    4,  ..., 2705, 2706, 2707]])

In [18]:
z = model.encode()
final_edge_index = model.decode_all(z)

In [40]:
final_edge_index[1]

tensor([   0,    1,    2,  ..., 2705, 2706, 2707])

In [27]:
print(final_edge_index[0][-2])
print(final_edge_index[1][-2])

tensor(2707)
tensor(2706)


In [37]:
final_edge_index[0]

tensor([   0,    0,    0,  ..., 2707, 2707, 2707])

### not important

In [60]:
import numpy as np
import pandas as pd

In [61]:
df = pd.read_json('ner_sent.json')

In [62]:
df

,Sentence_no,word_tag,only_tags,only_words,sentences
0,0,"[[ক্ষয়, B-SYM], [রোগ, O], [থেকে, O], [বাচ্চার,...","[B-SYM, O, O, B-GEN, O, O]","[ক্ষয়, রোগ, থেকে, বাচ্চার, উপায়, কি]",ক্ষয় রোগ থেকে বাচ্চার উপায় কি
1,1,"[[ধাতু, B-HBL], [দুর্বলতা, B-SYM]]","[B-HBL, B-SYM]","[ধাতু, দুর্বলতা]",ধাতু দুর্বলতা
2,2,"[[ক্ষয়, B-SYM], [রোগের, O], [সমাধান, O], [কি, O]]","[B-SYM, O, O, O]","[ক্ষয়, রোগের, সমাধান, কি]",ক্ষয় রোগের সমাধান কি
3,3,"[[আপু, B-GEN], [আমার, O], [গত, O], [মাস, O], [...","[B-GEN, O, O, O, O, B-MNS, O, B-SYM, B-INT, O]","[আপু, আমার, গত, মাস, ধরে, পিরিয়ড, হলে, ব্লিডিং...",আপু আমার গত মাস ধরে পিরিয়ড হলে ব্লিডিং বেশি হচ্ছে
4,4,"[[মাস, O], [পিরিয়ড, B-MNS], [হয়, O], [না, O], ...","[O, B-MNS, O, O, O, O, O, O, O, O, O, O]","[মাস, পিরিয়ড, হয়, না, অাবার, মাস, ঠিক, থাকে, অ...",মাস পিরিয়ড হয় না অাবার মাস ঠিক থাকে অাবার একি ...
...,...,...,...,...,...
661712,661712,"[[আমি, O], [অনেক, B-INT], [হস্তমৈতুন, B-ACN], ...","[O, B-INT, B-ACN, O, O, O, O, O, O, O, O, O]","[আমি, অনেক, হস্তমৈতুন, করি, তো, এখন, আমার, কি,...",আমি অনেক হস্তমৈতুন করি তো এখন আমার কি কোন প্রব...
661713,661713,"[[আমি, O], [অনেক, B-INT], [আগে, O], [থেকেই, O]...","[O, B-INT, O, O, B-EXO, B-INO, O, O, O, O, O, ...","[আমি, অনেক, আগে, থেকেই, হাত, মাড়ি, কিনতু, এখন,...",আমি অনেক আগে থেকেই হাত মাড়ি কিনতু এখন আমি কোন ...
661714,661714,"[[হস্তমৈথুন, B-ACN], [না, O], [করার, O], [ও, O...","[B-ACN, O, O, O, O, O]","[হস্তমৈথুন, না, করার, ও, পচায়, কি]",হস্তমৈথুন না করার ও পচায় কি
661715,661715,"[[বছর, O], [যাবত, O], [হস্তমৈথুন, B-ACN], [করছ...","[O, O, B-ACN, O, O, O, O, O, O, O]","[বছর, যাবত, হস্তমৈথুন, করছি, এখনো, ছাড়তে, পারছ...",বছর যাবত হস্তমৈথুন করছি এখনো ছাড়তে পারছি না কি...
